# WeatherPy
----

In [1]:
# Dependencies and Setup
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time

from citipy import citipy

from api_JAK import api_key

# CSV with collected data
output_data_file = 'output_data/cities.csv'

# Set ranges of potential latitudes and longitudes
lat_set = (-90, 90)
long_set = (-180, 180)

## Generate Cities List

In [3]:
# create emply lists for coordinates, cities/searchable city names, and country codes
lat_longs = []
cities = []
search_cities = [] # replaces each :space: in city name with :+: symbol allowing for usage in search web address
countries = []

# Create a set (min 500 :RESULTS:) of random coordinates
lats = np.random.uniform(low = -90.0000, high = 90.0000, size = 100)
longs = np.random.uniform(low = -180.0000, high = 180.0000, size = 100)
lat_longs = zip(lats, longs)

# Identify nearest city for each set of coordinates, add only if not already in list
for lat_long in lat_longs:
    city_name = citipy.nearest_city(lat_long[0], lat_long[1]).city_name
    country_code = citipy.nearest_city(lat_long[0], lat_long[1]).country_code
    if city_name not in cities:
        cities.append(city_name)
        countries.append(country_code)

# Loop to generate items for search_cities list. 
# Replace spaces in multi-word city names (so the API pulls address correctly)
for city in cities:
        readble_city = city.replace(' ', '+')
        search_cities.append(readble_city)

# generate searchable item list for API
search_list = list(zip(search_cities, countries))

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [ ]:
# Base API url for calls
url = 'http://api.openweathermap.org/data/2.5/weather?appid='
units = 'imperial'

In [6]:
# # # # # # # # # # # # # # # # # #
#   API ADDRESS/JSON TEST BLOCK   #
# # # # # # # # # # # # # # # # # #

## review data retrieved in call
# city = 'Orlando'
# country = 'US'
## replace both of these with proper variables in iterable code

# city_url = url + api_key + '&units=' + units + '&q=' + city + ',' + country
# response = requests.get(city_url) 
# weather_data = response.json() 

## test URL retrieval
# print(response.url)

## test dumps for reading json retrieval
# print(json.dumps(weather_data, indent=3, sort_keys=True))

In [ ]:
city_cap = [] # duplicates info from cities variable, but in a nicer format (proper capitalization)
country_cap = [] # duplicates info from countries variable, but in a nicer format (proper capitalization)
lat = []
temp = []
humidity = []
cloudiness = []
wind_speed = []
date = []
failure_count = []

call_count = 0

print('Processing Records\n=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=\n')

for call_count in range(len(search_list)):
    try:
        city_url = (url + api_key + '&units='+ units + '&q=' + search_list[call_count][0] + ',' + search_list[call_count][1])
        weather_data = requests.get(city_url).json() 

        if weather_data["cod"] == '404':
            call_count += 1
            failure_count.append(f'City Call {call_count}')
            print(f'City Call {call_count}: ***** FAILED: City Not Found *****')
        else:
            city_cap.append(weather_data["name"])
            city_log = weather_data["name"]
            country_cap.append(weather_data["sys"]["country"])
            country_log = weather_data["sys"]["country"]
            lat.append(weather_data["coord"]["lat"])
            temp.append(weather_data["main"]["temp_max"])
            humidity.append(weather_data["main"]["humidity"])
            cloudiness.append(weather_data["clouds"]["all"])
            wind_speed.append(weather_data["wind"]["speed"])
            date.append(weather_data["dt"])

            print(f'City Call {call_count + 1}:  {city_log}, {country_log}')

        print(city_url)
        call_count += 1

    except:
        print('= + = + = + = + = + = +     ERROR     + = + = + = + = + = + =')
    pass  ## trying pass instead of continue

print('Data Collection Complete!\n=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=')

In [ ]:
# quick check of list data
print(f'{len(search_list)} random cities generated.\n')
print(f'{len(failure_count)} FAILURES:\n{failure_count}\n')
# print(failure_count)
print((f'''Complete data will have {len(search_list) - len(failure_count)} items.

Item Count:
{len(city_cap)}
{len(country_cap)}
{len(lat)}
{len(temp)}
{len(humidity)}
{len(cloudiness)}
{len(wind_speed)}
{len(date)}'''))

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [ ]:
city_weather_df = (pd.DataFrame({'City': city_cap, 
                                 'Country': country_cap, 
                                 'Latitude': lat, 
                                 'Temperature (F)': temp, 
                                 'Humidity (%)': humidity, 
                                 'Cloudiness (%)': cloudiness, 
                                 'Wind Speed (mph)': wind_speed, 
                                 'Date': date}))
city_weather_df.head()

In [ ]:
city_weather_df.count()

### Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

#### Latitude vs. Temperature Plot

In [ ]:
## insert a heavier line at lat/x = 0 to indicate EQUATOR

plt.figure(figsize = (15, 10))

plt.scatter(city_weather_df['Latitude'], city_weather_df['Temperature (F)'], 
            marker = 'o', 
            color = 'skyblue', 
            edgecolor = 'teal', 
            alpha = 0.75)

# Title the plot.
plt.title('Latitude vs Temperature', fontsize = 30)

# Label and limits for x-axis.
plt.xlabel('Location: Degrees of Latitude', fontsize = 16)
plt.xlim (-90, 90)

# Label and limits for y-axis.
plt.ylabel('Temperature: °Fahrenheit', fontsize = 16)
plt.ylim (0, 115)

# Add light grid.
plt.grid(color = 'silver', linestyle = '-.', linewidth = 0.5)

plt.show()

#### Latitude vs. Humidity Plot

In [ ]:
## insert a heavier line at lat/x = 0 to indicate EQUATOR
## insert a colored line at y = to indicate 0

plt.figure(figsize = (15, 10))

plt.scatter(city_weather_df['Latitude'], city_weather_df['Humidity (%)'], 
            marker = 'o', 
            color = 'skyblue', 
            edgecolor = 'teal', 
            alpha = 0.75)

# Title the plot.
plt.title('Latitude vs Humidity', fontsize = 30)

# Label and limits for x-axis.
plt.xlabel('Location: Degrees of Latitude', fontsize = 16)
plt.xlim (-90, 90)

# Label and limits for y-axis.
plt.ylabel('Percent Humidity', fontsize = 16)
plt.ylim (-5, 105)

# Add light grid.
plt.grid(color = 'silver', linestyle = '-.', linewidth = 0.5)

plt.show()

#### Latitude vs. Cloudiness Plot

In [ ]:
## insert a heavier line at lat/x = 0 to indicate EQUATOR
## insert a colored line at y = to indicate 0

plt.figure(figsize = (15, 10))

plt.scatter(city_weather_df['Latitude'], city_weather_df['Cloudiness (%)'], 
            marker = 'o', 
            color = 'skyblue', 
            edgecolor = 'teal', 
            alpha = 0.75)

# Title the plot.
plt.title('Latitude vs Cloudiness', fontsize = 30)

# Label and limits for x-axis.
plt.xlabel('Location: Degrees of Latitude', fontsize = 16)
plt.xlim (-90, 90)

# Label and limits for y-axis.
plt.ylabel('Percent Cloudiness', fontsize = 16)
plt.ylim (-5, 105)

# Add light grid.
plt.grid(color = 'silver', linestyle = '-.', linewidth = 0.5)

plt.show()

#### Latitude vs. Wind Speed Plot

In [ ]:
plt.figure(figsize = (15, 10))

plt.scatter(city_weather_df['Latitude'], city_weather_df['Wind Speed (mph)'], 
            marker = 'o', 
            color = 'skyblue', 
            edgecolor = 'teal', 
            alpha = 0.75)

# Title the plot.
plt.title('Latitude vs Wind Speed', fontsize = 30)

# Label and limits for x-axis.
plt.xlabel('Location: Degrees of Latitude', fontsize = 16)
plt.xlim (-90, 90)

# Label and limits for y-axis.
plt.ylabel('Wind Speed (mph)', fontsize = 16)
plt.ylim (-5, 40)

# Add light grid.
plt.grid(color = 'silver', linestyle = '-.', linewidth = 0.5)

plt.show()